In [2]:
# OS/ User specific 
import os
#TRAINING_DATASET_PATH = "/home/jcharlet/workspace/datascience/machine_learning_notes/data" + '/train.csv'
TRAINING_DATASET_PATH = os.path.abspath(os.path.join( os.getcwd() , 'data')) + r'\train.csv'

### Kaggle project - Table of Contents
* 1 - Define the problem
* [Load data and displaying info](#load-data)
* 2 - Prepare Data
    * Identify features
        * Separate numerical from categorical features
        * Separate nominal and ordinal (from categorical features)
    * Clean data
        * Remove numerical features with missing values
        * Remove categorical features with missing values
        * drop outliers in numerical values # WIP
    * transform # TODO
        * transform categorical values #TODO
* 2 - Feature selection #WIP
    * Select features using random forest classifier #WIP
    
* 3 - Spot Check Algorithms
    * split dataset
    * train on multiple algorithms  # WIP

# Define the problem

From project https://www.kaggle.com/c/house-prices-advanced-regression-techniques

- Step 1: What is the problem? 

house prices are hard to predict, we want to know how much my beautiful flat is worth on the market in the USA

- Step 2: Why does the problem need to be solved? 

MONEY MONEY
and especially we need to learn Machine Learning

- Step 3: How would I solve the problem? 

housing agents do statistical studies.

Data pre-processing is very important. We have a lot of features, dirty data (missing values), numerical/ordinal/nominal values.

We'll follow the process from https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

- 1 - Define the Problem
- 2 - Prepare Data
- 3 - Spot Check Algorithms
- 4 - Improve Results
- 5 - Present Results

# Load data <a class="anchor" id="load-data"></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv(TRAINING_DATASET_PATH)


In [5]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

# Prepare data <a class="anchor" id="clean-data"></a>
## Identify features
### Separate numerical from categorical features

In [6]:
# Separating data features according to data type: numeric or string

X_numeric_labels = set(df._get_numeric_data().columns.tolist())
X_categorical_labels = set(df.columns.tolist()).difference(X_numeric_labels)

In [7]:
print('These are numerical features {} \n'.format(X_numeric_labels))
print('These are categorical features {} \n'.format(X_categorical_labels))


These are numerical features {'MasVnrArea', '1stFlrSF', 'Id', '2ndFlrSF', 'LotArea', 'FullBath', 'YearRemodAdd', 'BsmtHalfBath', 'GarageCars', 'GarageArea', 'BsmtUnfSF', 'HalfBath', 'SalePrice', 'KitchenAbvGr', 'MSSubClass', 'MoSold', 'BsmtFinSF2', 'YrSold', 'ScreenPorch', 'BsmtFullBath', 'LowQualFinSF', 'WoodDeckSF', 'PoolArea', 'LotFrontage', 'OverallQual', 'MiscVal', 'GarageYrBlt', 'OverallCond', 'Fireplaces', '3SsnPorch', 'YearBuilt', 'TotalBsmtSF', 'TotRmsAbvGrd', 'BsmtFinSF1', 'OpenPorchSF', 'BedroomAbvGr', 'GrLivArea', 'EnclosedPorch'} 

These are categorical features {'BsmtCond', 'MiscFeature', 'Heating', 'GarageQual', 'LotShape', 'BsmtQual', 'Exterior1st', 'GarageCond', 'BsmtFinType2', 'PavedDrive', 'Condition2', 'Foundation', 'GarageFinish', 'BsmtFinType1', 'SaleCondition', 'Exterior2nd', 'Alley', 'SaleType', 'CentralAir', 'BsmtExposure', 'Neighborhood', 'HouseStyle', 'Electrical', 'BldgType', 'PoolQC', 'Condition1', 'LotConfig', 'LandSlope', 'Functional', 'RoofMatl', 'Heatin

### Separate nominal and ordinal (from categorical features)

In [8]:
# we are dropping any categorical feature with at least one null value, we can relax this requirement by changing df > 0 to 
# df > n with n > 0

categorical_missing_values_labels = df[list(X_categorical_labels)].isnull().sum().loc[lambda df: df > 0].index.tolist()

In [9]:
categorical_missing_values_labels

['BsmtCond',
 'MiscFeature',
 'GarageQual',
 'BsmtQual',
 'GarageCond',
 'BsmtFinType2',
 'GarageFinish',
 'BsmtFinType1',
 'Alley',
 'BsmtExposure',
 'Electrical',
 'PoolQC',
 'MasVnrType',
 'Fence',
 'FireplaceQu',
 'GarageType']

In [10]:
print(" Number of Total Categorical Values = {} \n Number of Categorical Missing Values = {}  \n Difference = {}".format(len(X_categorical_labels), len(categorical_missing_values_labels), len(X_categorical_labels)- len(categorical_missing_values_labels) ))

 Number of Total Categorical Values = 43 
 Number of Categorical Missing Values = 16  
 Difference = 27


In [11]:
# Separating categorical features in two sets: nominal and ordinal
# The values in features asociated with categorical ordinal feautures are: [Ex, Gd, GLQ, GdPrv, Fin] ... etc

categorical_ordinal_list_values = ['Ex', 'Gd', 'GLQ', 'GdPrv', 'Fin'] # There are not all of the values. Here must be
# included all the possible values for the categorical ordinal features

X_categorical_nominal_labels = (df[list(X_categorical_labels)].drop(categorical_missing_values_labels,
        1).isin(categorical_ordinal_list_values) == False).all().loc[lambda df: df.values == True].axes

# The previous instruction returns the index of the categorical_nominal_values. I think we could include the following features:
#LandSlope has values: Gtl -> Gentle slopen; Mod -> Moderate Slope; Sev -> Severe Slope
#Functional has values: Typ -> Typical Functionality; Min1 -> Minor Deductions 1; Min2 -> Minor Deductions 2; Mod -> Moderate Deductions; Maj1 -> Major Deductions 1; Maj2 -> Major Deductions 2; Sev -> Severely Damaged; Sal -> Salvage only
#PavedDrive has values:   Y -> Paved ; P -> Partial Pavement; N -> Dirt/Gravel



In [12]:
print(X_categorical_nominal_labels)
print()
print(X_categorical_nominal_labels[0].tolist())

[Index(['Heating', 'LotShape', 'Exterior1st', 'PavedDrive', 'Condition2',
       'Foundation', 'SaleCondition', 'Exterior2nd', 'SaleType', 'CentralAir',
       'Neighborhood', 'HouseStyle', 'BldgType', 'Condition1', 'LotConfig',
       'LandSlope', 'Functional', 'RoofMatl', 'LandContour', 'RoofStyle',
       'Street', 'Utilities', 'MSZoning'],
      dtype='object')]

['Heating', 'LotShape', 'Exterior1st', 'PavedDrive', 'Condition2', 'Foundation', 'SaleCondition', 'Exterior2nd', 'SaleType', 'CentralAir', 'Neighborhood', 'HouseStyle', 'BldgType', 'Condition1', 'LotConfig', 'LandSlope', 'Functional', 'RoofMatl', 'LandContour', 'RoofStyle', 'Street', 'Utilities', 'MSZoning']


### Remarks
<p> From the set of set of **X_categorical_labels** the set of **X_categorical_nominal_labels** is extracted to get 
the set of **X_categorical_ordinal_labels**</p>

* On the set of **X_categorical_ordinal_labels** we can apply pag 104 **mapping strategy**

* On the set of **X_categorical_nominal_labels** we can apply **one-hot encoding strategy** pag 106

In [13]:
X_categorical_ordinal_labels = set(X_categorical_labels).difference(set(X_categorical_nominal_labels[0].tolist()))

In [14]:
# print(X_categorical_labels)

In [15]:
# df[list(X_categorical_ordinal_labels)]

## Clean data
### Remove numerical features with missing values

#### Some considerations


* <p> Page 107 of "our book":  "After executing the preceding code, the first column of the NumPy array X now holds the new
colour values, which are encoded as follows:
blue to 0
green to 1
red to 2
If we stop at this point and feed the array to our classifier, *we will make one of the most common 
mistakes in dealing with categorical data*. Can you spot the problem? *
Although the colour values do not come in any particular order, a learning algorithm will now assume
that green is larger than blue, and red is larger than green. 
Although this assumption is incorrect, the algorithm could still produce useful results. 
However, those results would not be optimal* "</p>

* <p> In the numerical feature variables there is the possibility that there are categorical numerical features variables. How to identify this kinf of features ? 
 I think the main problem is what strategy to use to replace values ? median, mode or mean ? </p>
 

In [ ]:
df_cleaned=df.copy()


#Let's check how many numerical features have missing values

# print(df[df._get_numeric_data().columns.tolist()].isnull().sum())

# Dropping columns: ['Id', 'LotFrontage', 'GarageYrBlt', 'SalePrice',] from numerical features

# dropping Id and 
df_cleaned= df_cleaned._get_numeric_data().drop(['Id'],1)
X_numerical_not_missing_values_labels=df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()
df_cleaned=df[X_numerical_not_missing_values_labels]
# df_cleaned = df_cleaned._get_numeric_data().drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'],1)

# print(df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum())

### Remove categorical features with missing values
### We need to change df_cleaned here !!

In [24]:
# Checking for missing values in categorical values in the data set

# print(df[list(X_categorical_labels)].isnull().sum())

# Dropping categorical features with missing values
# The following instruction returns the list of features with no missing values

#X_categorical_not_missing_values_labels = df[list(X_categorical_labels)].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()

df[list(X_categorical_labels)].drop(X_categorical_not_missing_values_labels, 1).columns.tolist()
print(len(df[list(X_categorical_labels)].drop(categorical_missing_values_labels, 1).columns.tolist()))
df_cleaned=df[X_categorical_not_missing_values_labels]



27


### Drop outliers in numerical values # WIP

In [ ]:
def drop_outliers(df, field_name):
    distance = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.percentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.percentile(df[field_name], 25) - distance].index, inplace=True)

# drop_outliers(df_cleaned, 'LotArea')    
df_cleaned_no_outlier = df.copy()
for feature in X_numerical_not_missing_values_labels:
    drop_outliers(df_cleaned_no_outlier, feature)    

In [ ]:
print("nb of rows with outliers:{}, without: {}".format(df.shape[0],df_cleaned_no_outlier.shape[0]))


We are currently removing far too many items, to review later.
can be done with sklearn anyway http://scikit-learn.org/stable/modules/outlier_detection.html

# Feature Selection

In [ ]:
# feature_columns=filter(lambda x: x !="SalePrice" and x!="Id", df.columns)
# feature_numerical_columns = df[feature_columns].select_dtypes(include=[np.number]).columns.tolist()
feature_numerical_columns=X_numerical_not_missing_values_labels
prediction_column="SalePrice"
# # print(feature_columns)
X = df[feature_numerical_columns]
Y=df[[prediction_column]];

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();
model = model.fit(X, Y)    
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("label {} : {}".format(feature_numerical_columns[f],importances[indices[f]]))
# print(indices)

# plt.bar(range(X.shape[1]),importances[indices],align='center')
# plt.xticks(range(X.shape[1]),feature_numerical_columns,rotation=90);
# plt.rcParams["figure.figsize"] = (17,5)


To draw graph and select features

# Spot Check Algorithms
## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = df[X_numerical_not_missing_values_labels]
Y=df[["SalePrice"]];
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)
print(Y_train.head())
X_train.head()

## Train on multiple models

In [ ]:

#training
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 

from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing

models = []
models.append(("SVC",SVC()))
models.append(("LinearSVC",LinearSVC()))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(("DecisionTree",DecisionTreeClassifier()))
models.append(("RandomForest",RandomForestClassifier()))

modelsWhichNeedNormalization = []
modelsWhichNeedNormalization.append(("LogisticRegression",LogisticRegression()))
modelsWhichNeedNormalization.append(("MLPClassifier",MLPClassifier(solver='lbfgs', random_state=0)))
modelsWhichNeedNormalization.append(("LinearRegression", LinearRegression()))
# models

In [ ]:
print("5 expected values: {}\n".format(Y_test[0:5].values))
for name,model in models:
#     Train the model using the training sets
    model.fit(X_train, Y_train)

    # Make predictions using the testing set
    y_pre = model.predict(X_test)
    print("Training on {}\n   mean squared log error: {}\n".format(name,mean_squared_log_error(Y_test, y_pre)))
    print("   5 predicted values: {}".format(y_pre[0:5]))

In [ ]:

# for name,model in modelsWhichNeedNormalization:
# #     Train the model using the training sets
#     X_train_scaled = preprocessing.scale(X_train)
#     model.fit(X_train_scaled, Y_train)

#     # Make predictions using the testing set
#     X_test_scaled = preprocessing.scale(X_test)
#     y_pre = model.predict(X_test_scaled)
#     print("Training on {}: {}".format(name,mean_squared_log_error(Y_test, y_pre)))

# crash on Jeremie's laptop